In [25]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [11]:
movie_data = pd.read_csv('data/movies_grouplens/movies.dat', delimiter='::', header=None, engine='python', encoding='latin1')
rating_data = pd.read_csv('data/movies_grouplens/ratings.dat', delimiter='::', header=None, engine='python', encoding='latin1')
user_data = pd.read_csv('data/movies_grouplens/users.dat', delimiter='::', header=None, engine='python', encoding='latin1')

In [12]:
movie_data.columns = ['MovieID', 'Title', 'Genres']
rating_data.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
user_data.columns = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

In [13]:
movie_data['Genres'] = movie_data['Genres'].fillna('')

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movie_data['Genres'])

In [14]:
user_movie_data = rating_data.merge(movie_data, on='MovieID')

In [15]:
movie_index = pd.Series(movie_data.index, index=movie_data['MovieID']).drop_duplicates()

In [16]:
user_profiles = {}

In [17]:
for user_id in rating_data['UserID'].unique():
    user_ratings = user_movie_data[user_movie_data['UserID'] == user_id]
    
    tfidf_weights = tfidf_matrix[user_ratings['MovieID'].apply(lambda x: movie_index[x])]
    user_profile = np.dot(user_ratings['Rating'], tfidf_weights.toarray()) / user_ratings['Rating'].sum()
    
    user_profile = user_profile / np.linalg.norm(user_profile)
    
    user_profiles[user_id] = user_profile

In [19]:
def recommend_movies_for_user(user_id, top_n=10):
    user_profile = user_profiles[user_id]
    
    sim_scores = cosine_similarity([user_profile], tfidf_matrix)[0]
    
    movie_indices = sim_scores.argsort()[-top_n:][::-1]
    
    return movie_data['Title'].iloc[movie_indices]

In [ ]:
def calculate_mse_for_user(user_id):
    user_ratings = user_movie_data[user_movie_data['UserID'] == user_id]
    mse_scores = []
    
    for movie_id in user_ratings['MovieID']:
        movie_tfidf = tfidf_matrix[movie_index[movie_id]].toarray()[0]
        user_profile = user_profiles[user_id]
        
        mse = mean_squared_error(user_profile, movie_tfidf)
        mse_scores.append(mse)
    
    return np.mean(mse_scores)

In [24]:
user_id = 181
print(f"Recommendations for User {user_id}:")
print(recommend_movies_for_user(user_id))

Recommendations for User 181:
1847                                    Buffalo 66 (1998)
20                                      Get Shorty (1995)
386                   Faster Pussycat! Kill! Kill! (1965)
1445                                      Best Men (1997)
1931                                 Lethal Weapon (1987)
1932                               Lethal Weapon 2 (1989)
1933                               Lethal Weapon 3 (1992)
1849                               Lethal Weapon 4 (1998)
3197    Man Bites Dog (C'est arrivé près de chez vous)...
3115                                       Montana (1998)
Name: Title, dtype: object
